In [ ]:
#pip install gymnasium[mujoco]

In [ ]:
import gymnasium as gym
import numpy as np
import torch
from torch import nn, optim
from time import sleep
from matplotlib.pyplot import plot
from IPython.display import clear_output

num_envs = 16
#base_env = gym.vector.SyncVectorEnv([lambda: gym.make("InvertedPendulum-v4") for _ in range(num_envs)])
base_env = gym.vector.SyncVectorEnv([lambda: gym.make("InvertedDoublePendulum-v4") for _ in range(num_envs)])

#wrapper to reset environments when episodes are done
class AutoResetVecEnv(gym.vector.VectorEnv):
    def __init__(self, venv):
        self.venv = venv
        self.num_envs = venv.num_envs
        self.observation_space = venv.observation_space
        self.action_space = venv.action_space

    def reset(self, **kwargs):
        return self.venv.reset(**kwargs)

    def step(self, actions):
        obs, reward, terminated, truncated, info = self.venv.step(actions)
        done_flags = np.logical_or(terminated, truncated)
        old_done_flags = done_flags.copy()
        if done_flags.any():
            new_obs, new_info = self.venv.reset()
            obs[done_flags] = new_obs[done_flags]
        return obs, reward, old_done_flags, old_done_flags, info

    def render(self):
        return self.venv.render()

    def close(self):
        self.venv.close()

#apply wrapper to environment
env = AutoResetVecEnv(base_env)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#policy network class: outputs mean and std of Gaussian action distribution
class PolicyNetwork(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.mean = nn.Linear(64, act_dim)
        self.log_std = nn.Parameter(torch.zeros(act_dim))

    def forward(self, x):
        x = self.fc(x)
        mean = self.mean(x)
        std = torch.exp(self.log_std)
        return mean, std

#value network: estimate state value for advantage estimation
class ValueNetwork(nn.Module):
    def __init__(self, obs_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

#get the dimensions of observation and action
obs_dim = env.observation_space.shape[1]
act_dim = env.action_space.shape[1]

pi = PolicyNetwork(obs_dim, act_dim).to(device)
V  = ValueNetwork(obs_dim).to(device)
optimizer = optim.Adam(list(pi.parameters()) + list(V.parameters()), lr=5e-4)

gamma = 0.99
lam = 0.95
num_updates = 100
T = 128
n_epochs = 5
mini_batch_size = 32
entropy_coef = 0.01

episode_rewards = np.zeros(num_envs)
completed_rewards = []

obs, info = env.reset()
obs = torch.tensor(obs, dtype=torch.float32, device=device)

for update in range(num_updates):
    states, actions, rewards, dones, log_probs, values = [], [], [], [], [], []

    #loop through time steps
    for t in range(T):
        state = obs #shape: [num_envs, obs_dim]
        with torch.no_grad():
            #get action dist and value prediction without updating gradient
            mean, std = pi(state) #mean, std: [num_envs, act_dim]
            dist = torch.distributions.Normal(mean, std)
            action = dist.sample()    #shape: [num_envs, act_dim]
            log_prob = dist.log_prob(action).sum(dim=-1)
            value = V(state).squeeze(-1)  #shape: [num_envs]

        #store states, actions, log_probs, value
        #each is a list of T tensors
        states.append(state)
        actions.append(action)
        log_probs.append(log_prob)
        values.append(value)

        #steps in parallel env
        actions_np = action.cpu().numpy()
        obs_np, reward, terminated, truncated, info = env.step(actions_np)
        done_flags = np.logical_or(terminated, truncated) #shape: [num_envs]

        #reward tracking
        episode_rewards += reward
        for i in range(num_envs):
            if done_flags[i]:
                completed_rewards.append(episode_rewards[i])
                episode_rewards[i] = 0.0

        #append reward/done as tensors with shape [num_envs]
        rewards.append(torch.tensor(reward, dtype=torch.float32, device=device))
        dones.append(torch.tensor(done_flags, dtype=torch.float32, device=device))

        #update current obs
        obs = torch.tensor(obs_np, dtype=torch.float32, device=device)

    #estimate value of final state
    with torch.no_grad():
        last_value = V(obs).squeeze(-1)
    values.append(last_value) #values has length T+1

    #convert collected data to tensors
    rewards_tensor = torch.stack(rewards, dim=0)
    dones_tensor   = torch.stack(dones,   dim=0)
    values_tensor  = torch.stack(values,  dim=0)

    #initialize GAE with zeros
    gae = torch.zeros(num_envs, device=device) #shape: [num_envs]
    adv_list = []
    #iterate backward through time to compute GAE
    for t in reversed(range(T)):
        # delta_t = r_t + gamma * V(s_{t+1}) * (1 - done_t) - V(s_t)
        delta = rewards_tensor[t] + gamma * values_tensor[t+1] * (1 - dones_tensor[t]) - values_tensor[t]
        # gae_t = delta_t + gamma * lambda * (1 - done_t) * gae_{t+1}
        gae = delta + gamma * lam * (1 - dones_tensor[t]) * gae
        adv_list.insert(0, gae) #insert front to maintain order
    advantages = torch.stack(adv_list, dim=0) #final advantage tensor with shape [T, num_envs]
    returns = advantages + values_tensor[:-1] # returns = advantages + baseline (value estimates)

    #flatten everything to [T*num_envs, ...]
    states_flat = torch.cat(states, dim=0)
    actions_flat = torch.cat(actions, dim=0)
    old_log_probs_flat = torch.cat(log_probs, dim=0)
    returns_flat = returns.view(-1)
    advantages_flat = advantages.view(-1)


    N = T * num_envs #total num samples
    for epoch in range(n_epochs):
        indices = torch.randperm(N)
        for i in range(0, N, mini_batch_size):
            idx = indices[i:i+mini_batch_size]
            s_batch = states_flat[idx]
            a_batch = actions_flat[idx]
            old_log_probs_batch = old_log_probs_flat[idx]
            ret_batch = returns_flat[idx]
            adv_batch = advantages_flat[idx]

            #policy distribution for batch
            mean, std = pi(s_batch)
            dist = torch.distributions.Normal(mean, std)
            new_log_probs = dist.log_prob(a_batch).sum(dim=-1)
            ratio = torch.exp(new_log_probs - old_log_probs_batch)

            #normalize advantages
            adv_norm = (adv_batch - adv_batch.mean()) / (adv_batch.std() + 1e-8)

            #PPO clipped objective
            surr1 = ratio * adv_norm
            surr2 = torch.clamp(ratio, 0.8, 1.2) * adv_norm
            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = nn.MSELoss()(V(s_batch).squeeze(-1), ret_batch)
            #entropy bonus
            entropy = dist.entropy().sum(dim=-1).mean()

            loss = policy_loss + value_loss - entropy_coef * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    if len(completed_rewards) > 0:
        running_avg = np.mean(completed_rewards[-10:])
    else:
        running_avg = 0.0
    print(f"Update {update}, Loss: {loss.item():.3f}, Running Avg Reward (last 10 eps): {running_avg:.1f}")

plot(completed_rewards)